In [1]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
import category_encoders as ce
from sklearn import preprocessing
import numpy as np
import random
import sys
import json
from operator import itemgetter as itg

Dict = []
FrequencyDict = {}
InfrequentDict = []
DataArray = []
TrueDict = {}

with open('AllSets.json') as json_data:
    data = json.load(json_data)

Rtext = []
lasttext = ["Blahblahblah"]
counter = 0
Dict.append("<PAD>")
TrueDict['<PAD>'] = counter
oglasttext = ''
print("Processing Stage 1")
for i, dataz in enumerate(data):
    print(dataz)
    for z, datazz in enumerate(data[dataz]['cards']):
        if 'text' in datazz:
            if oglasttext != datazz['text']:
                lasttext = datazz['text']
                oglasttext = datazz['text']
                lasttext = lasttext.replace("'", "")
                lasttext = lasttext.replace('"', "")
                lasttext = lasttext.replace(")", "")
                lasttext = lasttext.replace("(", "")
                lasttext = lasttext.replace("]", "")
                lasttext = lasttext.replace("[", "")
                lasttext = lasttext.replace("}", "")
                lasttext = lasttext.replace("{", "")
                lasttext = lasttext.replace(".", " <EOS> ")
                for word in lasttext.split():
                    if word not in FrequencyDict:
                        FrequencyDict[word] = 1
                    else:
                        FrequencyDict[word] = FrequencyDict[word] + 1
print("Processing Stage 2")
for i in FrequencyDict:
    if FrequencyDict[i] <= 20:
        if i.split()[0].istitle() == True:
            InfrequentDict.append(i)
print("Processing Stage 3")
for i, dataz in enumerate(data):
    #print(dataz)
    for z, datazz in enumerate(data[dataz]['cards']):
        if 'text' in datazz:
            if oglasttext != datazz['text']:
                lasttext = datazz['text']
                oglasttext = datazz['text']
                lasttext = lasttext.replace("'", "")
                lasttext = lasttext.replace('"', "")
                lasttext = lasttext.replace(")", "")
                lasttext = lasttext.replace("(", "")
                lasttext = lasttext.replace("]", "")
                lasttext = lasttext.replace("[", "")
                lasttext = lasttext.replace("}", "")
                lasttext = lasttext.replace("{", "")
                lasttext = lasttext.replace(".", " <EOS> ")
                for word in lasttext.split():
                    if word in InfrequentDict:
                        lasttext = lasttext.replace(word, " <NAME> ")
                DataArray.append(lasttext)
                for word in lasttext.split():
                    if word not in Dict:
                        Dict.append(word)
                        counter += 1
                        TrueDict[word] = counter
                        
print("Processing Stage 4")
#Dict.append("<EOS>")
MaxLen = 0
MaxStr = []
batch_size = 100
for l in range(len(DataArray)%batch_size):
    DataArray.pop()
for x, i in enumerate(DataArray):
    if len(DataArray[x].split()) > MaxLen:
        MaxLen = len(DataArray[x].split())
        MaxStr = DataArray[x]
print("Processing Stage 5")
BatchTensor = torch.zeros((int(len(DataArray)/batch_size),batch_size, MaxLen), device = 'cuda', dtype=torch.long) 
with torch.no_grad():
    for z in range(int(len(DataArray)/batch_size)):
        for x in range(batch_size):
            for l in range(len(DataArray[z*batch_size+x].split())):
                potato = TrueDict[DataArray[z*batch_size+x].split()[l]]
                BatchTensor[z][x][l] = potato      
            for b in range(MaxLen-l):
                if b != 0:
                    BatchTensor[z][x][l+b] = TrueDict["<PAD>"]

Processing Stage 1
10E
2ED
3ED
4ED
5DN
5ED
6ED
7ED
8ED
9ED
A25
AER
AKH
ALA
ALL
ANA
APC
ARB
ARC
ARN
ATH
ATQ
AVR
BBD
BFZ
BNG
BOK
BRB
BTD
C13
C14
C15
C16
C17
C18
CED
CEI
CHK
CHR
CM1
CM2
CMA
CMD
CN2
CNS
CON
CP1
CP2
CP3
CSP
CST
DD1
DD2
DDC
DDD
DDE
DDF
DDG
DDH
DDI
DDJ
DDK
DDL
DDM
DDN
DDO
DDP
DDQ
DDR
DDS
DDT
DDU
DGM
DIS
DKA
DKM
DOM
DPA
DRB
DRK
DST
DTK
DVD
E01
E02
EMA
EMN
EVE
EVG
EXO
EXP
F01
F02
F03
F04
F05
F06
F07
F08
F09
F10
F11
F12
F13
F14
F15
F16
F17
F18
FBB
FEM
FNM
FRF
FUT
G00
G01
G02
G03
G04
G05
G06
G07
G08
G09
G10
G11
G17
G18
G99
GK1
GK2
GNT
GPT
GRN
GS1
GTC
GVL
H09
H17
HHO
HML
HOP
HOU
HTR
HTR17
ICE
IMA
INV
ISD
ITP
J12
J13
J14
J15
J16
J17
J18
JGP
JOU
JUD
JVC
KLD
KTK
L12
L13
L14
L15
L16
L17
LEA
LEB
LEG
LGN
LRW
M10
M11
M12
M13
M14
M15
M19
MBS
MD1
ME1
ME2
ME3
ME4
MED
MGB
MIR
MM2
MM3
MMA
MMQ
MOR
MP2
MPR
MPS
MRD
NEM
NPH
OARC
OC13
OC14
OC15
OC16
OC17
OC18
OCM1
OCMD
ODY
OE01
OGW
OHOP
OLGC
ONS
OPC2
OPCA
ORI
OVNT
P02
P03
P04
P05
P06
P07
P08
P09
P10
P10E
P11
P15A
P2HG
PAER
PAKH
PAL00
PAL01
PAL02
P

In [5]:

AI = MTGAI(len(TrueDict), 100)
AI = AI.cuda()
LossF1 = nn.CrossEntropyLoss()
OptimF1 = optim.Adam(AI.parameters(), lr=.00003)
bestloss = 0
epochs = 100
lengths = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
lenph = []
indexes = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SortedData = torch.zeros((batch_size, MaxLen), device = 'cuda', dtype=torch.long)
firstiter= True
for epoch in range(epochs):
    totalloss = 0
    for i in range(int(len(DataArray)/batch_size)):
        datal = []
        AI.train()
        hidden = AI.init_hidden(batch_size)
        
        
        CardData = BatchTensor[i]
        CardData = CardData.cuda()
        for d in range(batch_size):
            lenph = [g for g in CardData[d] if g!=0]
            lengths[d] = len(lenph)
            indexes[d] = d
        targets = torch.zeros((batch_size, MaxLen), device = 'cuda', dtype=torch.long)
        ### Making the targets
        #print(lengths)
        #print(lengths.sort(reverse=True))
        #lengths, indexes = zip(*sorted(zip(lengths, indexes)))
        #print(lengths)
        #print(indexes)
        #targets = torch.zeros(MaxLen, max(lengths))
        #print(targets)
        for d in range(batch_size):
            #print(d)
            for x in range(lengths[d]-1):
                targets[d][x] = CardData[d][x+1]  
                # Set targets to be the next word in sentence
            #for z in range(MaxLen-lengths[d]-1):
                #targets[d][lengths[d]+z] = 0
                # Pad targets to be equal to padded data
        #for i in range(10):
           # print(CardData[i])    
            #print(targets[i])
        #datal = [(card_dataext, len([g for g in card_dataext if g!=0])) for card_dataext in CardData]
        #datasorted = sorted(datal, key=itg(1) , reverse=True)
        #CDS = torch.zeros((batch_size, MaxLen), device = 'cuda', dtype=torch.long)
        #LDS = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        #for bloopy in range(batch_size):
            #CDS[bloopy] = datasorted[bloopy][0]
            #LDS[bloopy] = datasorted[bloopy][1]
        #lengths.sort(reverse=True)
            #print(lengths)
        #print(CardData[0])
        #print(SortedData[0])
        
        ### Making the targets
            #CardData = (DataArray[i].split())[x]
            #NextData = (DataArray[i].split())[x+1]
            #NextData = torch.tensor(TrueDict[NextData], device='cuda')
            #NextData = NextData.unsqueeze(0)
        OptimF1.zero_grad()
        predictions, hidden = AI(CardData, hidden, batch_size, False)
        #for debug in range(10):
            #print(predictions.view(-1, len(Dict)))
            #print(len(predictions.view(-1, len(Dict))))
            #sys.exit()
            #print(len(predictions))
            #loss = LossF1(torch.narrow(predictions.view(-1, len(Dict)), 0, 0, lengths[x]), torch.narrow(targets[x], 0, 0, lengths[x]))
        for bloop in range(batch_size):
            #print(predictions[bloop][:lengths[bloop]])
            #print(targets[bloop][:lengths[bloop]])
            #print(len(predictions[bloop][:lengths[bloop]]))
            #print(len(targets[bloop][:lengths[bloop]]))
            loss = LossF1(predictions[bloop][:lengths[bloop]], targets[bloop][:lengths[bloop]])

            loss.backward(retain_graph=True)

            totalloss+= loss.item()
                        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            #torch.nn.utils.clip_grad_norm_(AI.parameters(), .1)

            OptimF1.step()
        if i != 0:
            print(predictions[0])
            print("Trained on ",i*batch_size, " cards.")
            print("Avg loss is: ",totalloss/(i*100))
            
            #total_loss += len(data) * loss.data
    print("Total loss this epoch:", totalloss)
    if bestloss > totalloss or bestloss==0:
        bestloss = totalloss
        print("New best loss! Val: ", bestloss)
        torch.save(AI.state_dict(), "./MTGAIbckup2.model.pth")

#print(TrueDict)
print("Done with training")

3398
tensor([[ 0.0141,  0.0396,  0.0312,  ..., -0.0765, -0.0230, -0.0750],
        [ 0.0691,  0.0557,  0.0208,  ..., -0.1150, -0.0376, -0.0866],
        [ 0.1230,  0.0656,  0.0226,  ..., -0.1419, -0.0471, -0.0967],
        ...,
        [ 0.2782,  0.0822,  0.0543,  ..., -0.2083, -0.0704, -0.1282],
        [ 0.2782,  0.0822,  0.0543,  ..., -0.2083, -0.0704, -0.1282],
        [ 0.2782,  0.0822,  0.0543,  ..., -0.2083, -0.0704, -0.1282]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  100  cards.
Avg loss is:  16.17511420249939
tensor([[ 0.0703,  0.0316,  0.0255,  ..., -0.1093, -0.0484, -0.1041],
        [ 0.2230,  0.0498,  0.0087,  ..., -0.1945, -0.0915, -0.1554],
        [ 0.4003,  0.0624,  0.0048,  ..., -0.2838, -0.1401, -0.2159],
        ...,
        [ 1.5216,  0.1270,  0.0315,  ..., -0.8144, -0.3943, -0.6210],
        [ 1.5216,  0.1270,  0.0315,  ..., -0.8144, -0.3943, -0.6210],
        [ 1.5216,  0.1270,  0.0315,  ..., -0.8144, -0.3943, -0.6210]],
       device='cuda:0

tensor([[ 1.9396, -0.9087,  0.2968,  ..., -1.8490, -1.5250, -1.8330],
        [ 4.7479, -2.2972,  0.6059,  ..., -4.3625, -3.6383, -4.4379],
        [ 5.0060, -2.4639,  0.6340,  ..., -4.5893, -3.7991, -4.6812],
        ...,
        [ 5.0763, -2.4685,  0.7223,  ..., -4.6974, -3.8956, -4.7997],
        [ 5.0763, -2.4685,  0.7222,  ..., -4.6975, -3.8956, -4.7997],
        [ 5.0763, -2.4684,  0.7222,  ..., -4.6976, -3.8956, -4.7998]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  1700  cards.
Avg loss is:  6.276127686220057
tensor([[ 1.9670, -0.8822,  0.4387,  ..., -1.8108, -1.4944, -1.7963],
        [ 5.0184, -2.3241,  0.9615,  ..., -4.4493, -3.7240, -4.5212],
        [ 5.3148, -2.5055,  1.0076,  ..., -4.7097, -3.9083, -4.7897],
        ...,
        [ 5.3807, -2.5188,  1.0917,  ..., -4.8126, -3.9971, -4.9072],
        [ 5.3807, -2.5188,  1.0917,  ..., -4.8127, -3.9971, -4.9073],
        [ 5.3807, -2.5187,  1.0918,  ..., -4.8128, -3.9971, -4.9074]],
       device='cuda:0', g

tensor([[ 4.3200, -0.9979,  0.8588,  ..., -4.6346, -4.1167, -4.6670],
        [ 5.0252, -1.1912,  0.9870,  ..., -5.6086, -4.9551, -5.7071],
        [ 5.0420, -1.1961,  0.9843,  ..., -5.6937, -4.9907, -5.7818],
        ...,
        [ 5.2741, -1.3028,  1.0801,  ..., -6.0017, -5.1574, -5.9862],
        [ 5.2740, -1.3028,  1.0802,  ..., -6.0018, -5.1574, -5.9863],
        [ 5.2740, -1.3027,  1.0802,  ..., -6.0018, -5.1574, -5.9863]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  3300  cards.
Avg loss is:  5.692899328506354
tensor([[ 4.1433, -1.0890,  0.7681,  ..., -4.7222, -4.2031, -4.7727],
        [ 4.7293, -1.2733,  0.8523,  ..., -5.6293, -4.9845, -5.7474],
        [ 4.7355, -1.2714,  0.8451,  ..., -5.7098, -5.0141, -5.8197],
        ...,
        [ 5.0122, -1.4279,  0.9505,  ..., -6.0280, -5.1995, -6.0708],
        [ 5.0122, -1.4278,  0.9505,  ..., -6.0280, -5.1995, -6.0709],
        [ 5.0122, -1.4278,  0.9505,  ..., -6.0280, -5.1996, -6.0709]],
       device='cuda:0', g

tensor([[ 4.0382, -1.3047,  1.4166,  ..., -4.4410, -3.9891, -4.4068],
        [ 5.2000, -1.7696,  1.8488,  ..., -6.1412, -5.5815, -6.2450],
        [ 5.2085, -1.7804,  1.8026,  ..., -6.2425, -5.6498, -6.3483],
        ...,
        [ 5.5454, -2.0543,  1.8313,  ..., -6.6896, -6.0591, -6.7155],
        [ 5.5455, -2.0544,  1.8314,  ..., -6.6896, -6.0591, -6.7155],
        [ 5.5455, -2.0545,  1.8314,  ..., -6.6896, -6.0592, -6.7156]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  4900  cards.
Avg loss is:  5.546763279973244
tensor([[ 4.9293, -0.7597,  1.8194,  ..., -5.7556, -5.2697, -5.8121],
        [ 5.0021, -0.7630,  1.8682,  ..., -6.1232, -5.6045, -6.2907],
        [ 5.0055, -0.7447,  1.8234,  ..., -6.1948, -5.6550, -6.3625],
        ...,
        [ 5.3446, -0.9315,  1.8510,  ..., -6.7273, -6.1317, -6.7744],
        [ 5.3447, -0.9316,  1.8510,  ..., -6.7273, -6.1318, -6.7745],
        [ 5.3447, -0.9317,  1.8510,  ..., -6.7273, -6.1318, -6.7746]],
       device='cuda:0', g

tensor([[ 4.3799, -1.7484,  0.6522,  ..., -5.8538, -5.2957, -5.7533],
        [ 4.3903, -1.7886,  0.5667,  ..., -6.7795, -6.0040, -6.8546],
        [ 4.3777, -1.7837,  0.4129,  ..., -6.9769, -6.1216, -7.0433],
        ...,
        [ 4.6851, -2.0648,  0.2334,  ..., -7.5589, -6.6937, -7.3986],
        [ 4.6851, -2.0650,  0.2333,  ..., -7.5589, -6.6939, -7.3987],
        [ 4.6852, -2.0651,  0.2333,  ..., -7.5590, -6.6940, -7.3988]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  6500  cards.
Avg loss is:  5.4700994077095615
tensor([[ 4.1217, -1.6676,  0.5976,  ..., -5.4244, -4.9122, -5.3262],
        [ 4.3967, -1.8487,  0.5152,  ..., -6.6954, -6.0041, -6.7889],
        [ 4.3803, -1.8301,  0.3533,  ..., -6.9096, -6.1329, -7.0229],
        ...,
        [ 4.6555, -2.0889,  0.1679,  ..., -7.5003, -6.7130, -7.3356],
        [ 4.6556, -2.0890,  0.1679,  ..., -7.5004, -6.7131, -7.3356],
        [ 4.6557, -2.0891,  0.1679,  ..., -7.5005, -6.7133, -7.3356]],
       device='cuda:0', 

...

tensor([[ -3.4161,  -7.7961,  -6.4687,  ..., -19.3477, -20.0672, -20.4173],
        [  8.4533,   3.2684,   0.4772,  ...,  -6.0732,  -7.7886,  -7.3364],
        [ -4.0178,  -5.0317,  -8.1873,  ..., -22.9575, -23.3560, -22.6950],
        ...,
        [-15.7223, -16.3085, -21.1309,  ..., -26.0513, -27.2065, -26.4966],
        [-15.7225, -16.3089, -21.1324,  ..., -26.0512, -27.2066, -26.4961],
        [-15.7227, -16.3093, -21.1339,  ..., -26.0510, -27.2068, -26.4957]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  35600  cards.
Avg loss is:  1.7954141216879982
tensor([[  1.1975,  -5.9398,  -3.3378,  ..., -19.0663, -19.8995, -20.2790],
        [ -9.5199,  -8.8479,  -9.0681,  ..., -23.4080, -24.4343, -24.3103],
        [  0.2923,  -0.0828,   1.7739,  ..., -14.6866, -15.0605, -14.8184],
        ...,
        [-15.4444, -17.1268, -21.9406,  ..., -26.4121, -27.5518, -27.1041],
        [-15.4453, -17.1258, -21.9406,  ..., -26.4111, -27.5509, -27.1032],
        [-15.4462, -17.1249,

tensor([[ -5.4938, -11.3639,  -7.8334,  ..., -26.0509, -25.7636, -25.7489],
        [-13.4804, -13.3872, -12.5392,  ..., -27.9815, -27.7238, -28.8857],
        [-14.1208, -13.8575, -13.8697,  ..., -31.5179, -32.5825, -32.9563],
        ...,
        [-15.6891, -16.2576, -20.3179,  ..., -25.3956, -26.7482, -25.8078],
        [-15.6884, -16.2562, -20.3165,  ..., -25.3951, -26.7473, -25.8066],
        [-15.6878, -16.2548, -20.3151,  ..., -25.3945, -26.7464, -25.8054]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  37100  cards.
Avg loss is:  1.7917259633898575
tensor([[ -8.4050, -11.0324,  -8.1492,  ..., -24.7393, -25.5782, -24.9442],
        [-10.9756, -12.7718, -10.5857,  ..., -27.5163, -28.4002, -27.8186],
        [  2.0639,  -0.5487,   0.3597,  ..., -16.1765, -16.3259, -16.9876],
        ...,
        [-15.4325, -16.8441, -20.8226,  ..., -25.9199, -27.4130, -26.2823],
        [-15.4320, -16.8422, -20.8213,  ..., -25.9196, -27.4124, -26.2814],
        [-15.4315, -16.8403,

tensor([[ -2.2848,  -9.7932,  -6.1644,  ..., -21.4232, -22.2448, -22.6972],
        [ -6.3903, -12.3288,  -7.0204,  ..., -25.1606, -25.7601, -25.4988],
        [-11.1090, -11.1293, -12.7614,  ..., -26.6699, -27.1119, -27.3269],
        ...,
        [-14.6321, -16.8980, -21.8549,  ..., -26.9233, -27.3720, -27.3498],
        [-14.6332, -16.8978, -21.8546,  ..., -26.9233, -27.3722, -27.3499],
        [-14.6344, -16.8976, -21.8543,  ..., -26.9232, -27.3724, -27.3500]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  38600  cards.
Avg loss is:  1.8049257379071546
tensor([[-10.4293, -10.1056,  -9.1375,  ..., -25.2310, -25.1031, -25.1039],
        [ -4.1258, -11.0579, -14.7200,  ..., -26.9529, -28.1336, -27.7273],
        [-14.1286, -17.2005, -16.1344,  ..., -26.1577, -26.5730, -27.6891],
        ...,
        [-15.5175, -18.5815, -22.3322,  ..., -25.9793, -26.8836, -26.3053],
        [-15.4825, -18.4904, -22.3303,  ..., -25.9556, -26.8439, -26.2956],
        [-15.5019, -18.4574,

tensor([[ -8.8911, -11.4356,  -9.7061,  ..., -24.6862, -25.3410, -24.8231],
        [-13.1785, -15.0462, -15.9270,  ..., -25.0752, -26.4184, -25.1203],
        [ -5.1853,  -8.8404, -10.4160,  ..., -23.9907, -24.2942, -23.7428],
        ...,
        [-16.1870, -18.1963, -21.5083,  ..., -26.1812, -27.8377, -26.1519],
        [-16.1866, -18.1949, -21.5076,  ..., -26.1810, -27.8375, -26.1513],
        [-16.1863, -18.1936, -21.5070,  ..., -26.1809, -27.8373, -26.1508]],
       device='cuda:0', grad_fn=<SelectBackward>)
Trained on  40100  cards.
Avg loss is:  1.8266459446261054
tensor([[ -9.0671, -11.4822,  -9.9897,  ..., -24.8370, -25.4892, -24.9586],
        [-13.4120, -15.3223, -15.8896,  ..., -25.3333, -26.6363, -25.3906],
        [ -8.0011, -13.1089, -12.3395,  ..., -21.7664, -21.7949, -21.1057],
        ...,
        [-16.2600, -17.5482, -20.8846,  ..., -25.3343, -26.8703, -25.1575],
        [-16.2606, -17.5484, -20.8857,  ..., -25.3348, -26.8707, -25.1576],
        [-16.2612, -17.5487,

KeyboardInterrupt: 

In [145]:
AI = MTGAI(len(TrueDict), 100)
AI = AI.cuda()
AI.load_state_dict(torch.load("./MTGAIbckup2.model.pth"))
hidden = AI.init_hidden(1, 1)
temperature = .65
num_words = 25
initword = Dict[random.randint(0, len(Dict))]
#initword = 'Target'
print(initword, '', end='')
initword = TrueDict[initword]
initword = torch.tensor(initword, device = 'cuda')

for i in range(num_words):
    #print(i)
    #print(Dict[9374]
    output, hidden = AI(initword, hidden, 1, True)
    word_weights = output.squeeze().data.div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    initword.data.fill_(word_idx)
    for searchword, searchid in TrueDict.items():
        if searchid == word_idx:
            word = searchword


    print(word + ' ', end='')
print("Done")

3398
Goblins — Whenever a land enters the battlefield under your control, <NAME> <NAME> deals damage equal to your hand <EOS> <PAD> creature gets +1/+1 until end Done


In [45]:
class MTGAI(nn.Module):
    
    def __init__(self, Dsize, input_size):
        super(MTGAI, self).__init__()
        self.Embed1 = nn.Embedding(Dsize, 1, padding_idx=0)
        self.GRU = nn.GRU(1, 125, 3, batch_first=True) # input size, hidden size, num of layers to use
        self.Drop1 = nn.Dropout(.5)
        self.Drop2 = nn.Dropout(.5)
        self.dsize = Dsize
        print(self.dsize)
        self.Lin1 = nn.Linear(125, self.dsize)
        #self.Lin1.bias.data.fill_(0)
        #self.Lin1.weight.data.uniform_(-.1, .1)
        self.input_size = input_size
        self.hidden = 0
        
    def forward(self, x, h, batch_size, test):
        output = []
        if test == False:
            y = self.Embed1(x)
            #print(y)
            #print(y.size())
            y, newh = self.GRU(y, h)
            #print(y.size())
            y = self.Lin1(y)
            #print(y.size())
            #for z in x:
                #y = self.Embed1(z).view(1, 1, -1)
                #y, newh = self.GRU(y, h)
                #y = self.Lin1(y)
                #output.append(y)
        if test == True:
            y = x
            y = y.unsqueeze(0)
            #print(y)
            y = self.Embed1(y)
            y = y.unsqueeze(0)
            y, newh = self.GRU(y, h)
            y = self.Lin1(y)
        return y, newh
    
    def init_hidden(self, batch_size, input_size=1):
        weight = next(self.parameters()).data
        return Variable(weight.new(3, batch_size, 125).zero_())